In [ ]:
import cv2

# Load the image
img = cv2.imread('"C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg"')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect edges using Canny edge detection
edges = cv2.Canny(gray, 100, 200)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Analyze the contours
num_scratches = 0
num_dents = 0
for contour in contours:
    # Calculate contour metrics
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    (x, y, w, h) = cv2.boundingRect(contour)
    aspect_ratio = w / float(h)
    
    # Check if contour corresponds to a scratch or a dent
    if area < 100 and aspect_ratio > 5:
        num_scratches += 1
    elif area > 1000 and aspect_ratio < 0.2:
        num_dents += 1

# Print the results
print("Number of scratches:", num_scratches)
print("Number of dents:", num_dents)

In [ ]:
import cv2

# Load the image
try:
    img = cv2.imread('C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg')
    if img is None:
        raise FileNotFoundError
except FileNotFoundError:
    print("Error: Unable to load image. Please check the file path and make sure the image file exists.")
    exit()

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect edges using Canny edge detection
edges = cv2.Canny(gray, 100, 200)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Analyze the contours
num_scratches = 0
num_dents = 0
for contour in contours:
    # Calculate contour metrics
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    (x, y, w, h) = cv2.boundingRect(contour)
    aspect_ratio = w / float(h)
    
    # Check if contour corresponds to a scratch or a dent
    if area < 100 and aspect_ratio > 5:
        num_scratches += 1
    elif area > 1000 and aspect_ratio < 0.2:
        num_dents += 1

# Print the results
print("Number of scratches:", num_scratches)
print("Number of dents:", num_dents)


In [ ]:
import cv2

# Load the image
try:
    img = cv2.imread('C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg')
    if img is None:
        raise FileNotFoundError
except FileNotFoundError:
    print("Error: Unable to load image. Please check the file path and make sure the image file exists.")
    exit()

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect edges using Canny edge detection
edges = cv2.Canny(gray, 100, 200)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Analyze the contours
num_scratches = 0
num_dents = 0
for contour in contours:
    # Calculate contour metrics
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    (x, y, w, h) = cv2.boundingRect(contour)
    aspect_ratio = w / float(h)
    
    # Check if contour corresponds to a scratch or a dent
    if area < 100 and aspect_ratio > 5:
        num_scratches += 1
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
    elif area > 1000 and aspect_ratio < 0.2:
        num_dents += 1
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with contours
cv2.imshow('Image with Contours', img)
cv2.waitKey(0)

# Print the results
print("Number of scratches:", num_scratches)
print("Number of dents:", num_dents)


In [ ]:
import cv2
import numpy as np

# Load YOLOv3 weights and configuration
net = cv2.dnn.readNetFromDarknet("C:\\Users\\chint\\anaconda3\\envs\\CV\\yolov3.cfg", 'C:\\Users\\chint\\anaconda3\\envs\\CV\\yolov3.weights')

# Set target classes
classes = ["scratch", "dent"]

# Load the image
img = cv2.imread("C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg")

# Set input image size and scale factor
inp_size = (416, 416)
scale_factor = 1 / 255.0

# Create a blob from the input image
blob = cv2.dnn.blobFromImage(img, scale_factor, inp_size, swapRB=True, crop=False)

# Set input blob as network input
net.setInput(blob)

layer_names = net.getLayerNames()

# Get unconnected output layer indices
output_layer_indices = net.getUnconnectedOutLayers()
output_layer_indices = [(i,) for i in output_layer_indices.tolist()]

# Check if the output_layer_indices array is not empty
if len(output_layer_indices) > 0:
    # Get output layer names
    output_layers = [layer_names[i[0] - 1] for i in output_layer_indices]
else:
    print("No output layers found")
    exit()

# Forward pass through network
outputs = net.forward(output_layers)

# Filter outputs by confidence threshold and class label
conf_threshold = 0.5
class_ids = []
confidences = []
boxes = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if class_id < len(classes) and confidence > conf_threshold and classes[class_id] in classes:
            center_x = int(detection[0] * img.shape[1])
            center_y = int(detection[1] * img.shape[0])
            w = int(detection[2] * img.shape[1])
            h = int(detection[3] * img.shape[0])
            x = center_x - w // 2
            y = center_y - h // 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

# Draw bounding boxes around the detected objects
for i in range(len(boxes)):
    (x, y, w, h) = boxes[i]
    label = f"{classes[class_ids[i]]} ({confidences[i]:.2f})"
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Count the number of scratches and dents
num_scratches = class_ids.count(classes.index('scratch'))
num_dents = class_ids.count(classes.index('dent'))

# Display the image with bounding boxes
cv2.imshow("Detected Scratches and Dents", img)
cv2.waitKey(0)

# Print the results
print("Number of scratches:", num_scratches)
print("Number of dents:", num_dents)


In [1]:
import cv2
import numpy as np

# Load the image
img = cv2.imread("C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold the image
ret, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Find contours in the image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around the detected contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', img)
#cv2.waitKey(0)


In [1]:
import cv2
import numpy as np

# Load the image
img = cv2.imread("C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 100, 200)

# Perform contour detection
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Define the minimum and maximum area thresholds for contours
min_area = 50
max_area = 10000

# Identify the contours that meet the area thresholds and have a rectangular shape
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > min_area and area < max_area:
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h
            if aspect_ratio > 0.2 and aspect_ratio < 5.0:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', img)
cv2.waitKey(0)


48

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread("C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 100, 200)

# Perform contour detection
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Define the minimum and maximum area thresholds for contours
min_area = 50
max_area = 10000

# Identify the contours that meet the area thresholds and have a rectangular shape
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > min_area and area < max_area:
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h
            if aspect_ratio > 0.2 and aspect_ratio < 5.0:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', img)
cv2.waitKey(0)


In [7]:
import cv2
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Set the path to the frozen inference graph file
model_path = 'faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'

# Load the frozen inference graph
with tf.gfile.GFile(model_path, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def = tf.GraphDef()
    od_graph_def.ParseFromString(serialized_graph)

# Set target classes
classes = ['dent']

# Load the pre-trained model into memory
with detection_graph.as_default():
    tf.import_graph_def(od_graph_def, name='')

# Load the pre-trained Faster R-CNN model
#model = cv2.dnn.readNetFromTensorflow('faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb', 'faster_rcnn_inception_v2_coco_2018_01_28/graph.pbtxt')

# Set target classes
classes = ['dent']

# Load the image
try:
    img = cv2.imread("C:\\Users\\chint\\OneDrive\\Desktop\\Hackathon\\BinaryBrew_Datahack\\test\\28.jpg")
    if img is None:
        raise FileNotFoundError
except FileNotFoundError:
    print("Error: Unable to load image. Please check the file path and make sure the image file exists.")
    exit()

# Set input image size and scale factor
inp_size = (600, 600)
scale_factor = 1 / 255.0

# Create a blob from the input image
blob = cv2.dnn.blobFromImage(img, scale_factor, inp_size, swapRB=True, crop=False)

# Set input blob as model input
model.setInput(blob)

# Get model output
outputs = model.forward()

# Filter outputs by confidence threshold and class label
conf_threshold = 0.5
class_ids = []
confidences = []
boxes = []
for output in outputs:
    for detection in output:
        class_id = int(detection[1])
        confidence = detection[2]
        if class_id < len(classes) and confidence > conf_threshold and classes[class_id] in classes:
            box = detection[3:7] * np.array([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
            (x, y, w, h) = box.astype("int")
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

# Draw bounding boxes around the detected objects
for i in range(len(boxes)):
    (x, y, w, h) = boxes[i]
    label = f"{classes[class_ids[i]]} ({confidences[i]:.2f})"
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Count the number of dents
num_dents = class_ids.count(classes.index('dent'))

# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', img)
cv2.waitKey(0)

# Print the results
print("Number of dents:", num_dents)


NotFoundError: NewRandomAccessFile failed to Create/Open: faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb : The system cannot find the path specified.
; No such process